## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [1]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.5
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset_normalizer, idna, urllib3
Required-by: anaconda-auth, anaconda-catalogs, anaconda-client, anaconda-project, conda, conda-build, conda-libmamba-solver, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, PyGithub, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [2]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.13.5
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve, typing-extensions
Required-by: conda-build, nbconvert


In [3]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup


In [5]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.5
bs4 버전 = 4.13.5


### 1. 뉴스 제목 추출하기

In [20]:
# IT/과학 뉴스 
#url = 'https://news.naver.com/section/105'

# dict타입으로 요청 파라미터 설정
req_param = {
    'sid': 105
}
# 
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보 ( 사람처럼 보이게 하기 위함 )
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers=req_header)
print(res.status_code)
print(res.ok)
print(type(res))
#print(res.text)
# 응답(response)이 OK 이면
# 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성  
if res.ok:
    soup = BeautifulSoup(res.text,'html.parser')
    print(len(soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")))
    # CSS 선택자를 사용해서 a tag 목록 가져오기
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
    print(type(a_tags), type(a_tags[0])) # [Tag,Tag]
    # <a> 태그 리스트 순회하기    
    for a_tag in a_tags:
        title = a_tag.text.strip()
        link = a_tag['href']
        print(title, link)
else:
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')

https://news.naver.com/section/105
200
True
<class 'requests.models.Response'>
88
<class 'bs4.element.ResultSet'> <class 'bs4.element.Tag'>
넷마블, 지난해 영업익 64% 증가…매출 역대 최대 https://n.news.naver.com/mnews/article/422/0000830752
 https://n.news.naver.com/mnews/article/comment/422/0000830752
최수연 "대형마트 규제 완화, 네이버에도 긍정적…커머스 투자 계속"[컨콜] https://n.news.naver.com/mnews/article/374/0000491169
 https://n.news.naver.com/mnews/article/comment/374/0000491169
초음파로 ‘암세포’ 타격…‘면역 깨우고 부작용 줄인’ 새 치료법 개발 https://n.news.naver.com/mnews/article/662/0000087991
 https://n.news.naver.com/mnews/article/comment/662/0000087991
네이버 "엔비디아 플랫폼 활용해 실외 로봇배송 실증 추진"[컨콜] https://n.news.naver.com/mnews/article/003/0013753256
 https://n.news.naver.com/mnews/article/comment/003/0013753256
초정밀 국내지도 반출될까…구글 보완서류 마감일 정부 제출 https://n.news.naver.com/mnews/article/025/0003501501
 https://n.news.naver.com/mnews/article/comment/025/0003501501
통신 업계 해킹 여파에 반사이익 본 LG유플, 무선 가입 3000만개 돌파 https://n.news.naver.com/mnews/article/009/0005632850


### 1.1 뉴스제목 추출하는 함수 선언하기

In [18]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section_name):  #print_new('생활/문화') 
    sid = section_dict.get(section_name,'정치')
    url = f'https://news.naver.com/section/{sid}'
    print(f'{section_name} 뉴스 {url}')
    req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
    }
    res = requests.get(url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text,'html.parser')
        # CSS 선택자를 사용해서 a tag 목록 가져오기
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
        # <a> 태그 리스트 순회하기    
        for a_tag in a_tags:
            title = a_tag.text.strip()
            link = a_tag['href']
            print(title, link)
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')

In [19]:
print_news('경제')

경제 뉴스 https://news.naver.com/section/101
KB금융, 순이익 5.8조원 사상 최대…역대 최대 주주환원 나선다 https://n.news.naver.com/mnews/article/014/0005474101
 https://n.news.naver.com/mnews/article/comment/014/0005474101
대통령이 ‘담합’ 때리자 설탕·밀가루 업체들 값 내린다 https://n.news.naver.com/mnews/article/005/0001830597
 https://n.news.naver.com/mnews/article/comment/005/0001830597
'연중무휴' 식자재마트에 밀리는 SSM 가맹점…규제 검토 목소리 https://n.news.naver.com/mnews/article/008/0005314637
 https://n.news.naver.com/mnews/article/comment/008/0005314637
쭉쭉 늘더니 국민연금마저 뛰어넘었다…서학개미 해외주식투자 450억달러(종합) https://n.news.naver.com/mnews/article/277/0005718044
 https://n.news.naver.com/mnews/article/comment/277/0005718044
[속보]코스피 매도사이드카 발동···장중 5% 급락해 5000선 깨지기도 https://n.news.naver.com/mnews/article/032/0003426268
 https://n.news.naver.com/mnews/article/comment/032/0003426268
"소음·탄소 줄인다"…배민, 배달 오토바이 10대 중 6대 전기로 https://n.news.naver.com/mnews/article/008/0005314585
 https://n.news.naver.com/mnews/article/comment/008/0005314585
개인정보위 "쿠팡 16만 5천 계정 추가 정보 유출 조사 

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [21]:
import requests
import os

# 육아일기 73회차
req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=812354&no=208&week=sun',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url, headers=req_header)
    print(res.status_code)        
    # binary 응답 데이터 가져오기
    img_data = res.content    
    # url에서 파일명만 추출하기
    file_name = os.path.basename(img_url)
    print(file_name)        
    # binday data를 file에 write하기
    with open(file_name,'wb') as file:
        print(f'Writing to {file_name}({len(img_data):,} bytes)')
        file.write(img_data)

200
20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_1.jpg
Writing to 20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_1.jpg(124,462 bytes)
200
20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_2.jpg
Writing to 20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_2.jpg(141,956 bytes)
200
20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_3.jpg
Writing to 20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_3.jpg(170,913 bytes)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [22]:
import requests
from bs4 import BeautifulSoup
webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=812354&no=208&week=thu'

req_header = {
    'referer':webtoon_url,
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
}

res = requests.get(webtoon_url, headers=req_header)
if res.ok:
    # .jpg 파일명을 추출해서 list에 저장
    img_url_list = []
else:
    print(f'Error Code = {res.status_code}')

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.



200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청


In [ ]:
# 캡차 이미지 요청


In [ ]:
#  사용자 입력값 검증 요청



* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


### 5. 블로그 검색하기

In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
